<a href="https://colab.research.google.com/github/gauravz7/RAG/blob/main/AdobePDF_RAG_Jan24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:

!pip install pdfservices-sdk

In [2]:
%%writefile pdfservices-api-credentials.json

{
 "client_credentials": {
  "client_id": "7ff5bf6cae1e49539df149143642659b",
  "client_secret": "p8e-3GEBMO81nXg8AV2mPkOPS-ThaVyaAr61"
 },
 "service_principal_credentials": {
  "organization_id": "87211D7265AE0D700A495C49@AdobeOrg"
 }
}

Writing pdfservices-api-credentials.json


In [10]:
os.environ['PDF_SERVICES_CLIENT_ID'] = "7ff5bf6cae1e49539df149143642659b"
os.environ['PDF_SERVICES_CLIENT_SECRET'] = "p8e-3GEBMO81nXg8AV2mPkOPS-ThaVyaAr61"

In [26]:
from adobe.pdfservices.operation.auth.credentials import Credentials
from adobe.pdfservices.operation.exception.exceptions import ServiceApiException, ServiceUsageException, SdkException
from adobe.pdfservices.operation.execution_context import ExecutionContext
from adobe.pdfservices.operation.io.file_ref import FileRef
from adobe.pdfservices.operation.pdfops.extract_pdf_operation import ExtractPDFOperation
from adobe.pdfservices.operation.pdfops.options.extractpdf.extract_pdf_options import ExtractPDFOptions
from adobe.pdfservices.operation.pdfops.options.extractpdf.extract_element_type import ExtractElementType
#from adobe.pdfservices.operation.pdfops.options.extractpdf.extract_pdf_options.ExtractPDFOptions import ExtractRenditionsElementType
#from adobe.pdfservices.operation.pdfops.options.extractpdf import TableStructureType

from adobe.pdfservices.operation.auth.credentials import Credentials
from adobe.pdfservices.operation.exception.exceptions import ServiceApiException, ServiceUsageException, SdkException
from adobe.pdfservices.operation.pdfops.options.extractpdf.extract_pdf_options import ExtractPDFOptions
from adobe.pdfservices.operation.pdfops.options.extractpdf.extract_element_type import ExtractElementType
from adobe.pdfservices.operation.pdfops.options.extractpdf.extract_renditions_element_type import \
    ExtractRenditionsElementType
from adobe.pdfservices.operation.pdfops.options.extractpdf.table_structure_type import TableStructureType
from adobe.pdfservices.operation.execution_context import ExecutionContext
from adobe.pdfservices.operation.io.file_ref import FileRef
from adobe.pdfservices.operation.pdfops.extract_pdf_operation import ExtractPDFOperation


import os.path
import zipfile
import json

In [7]:
input_pdf = "Geminipaper.pdf"

In [12]:
#Initial setup, create credentials instance.
credentials = Credentials.service_principal_credentials_builder().with_client_id(os.getenv('PDF_SERVICES_CLIENT_ID')).with_client_secret(os.getenv('PDF_SERVICES_CLIENT_SECRET')).build();

#Create an ExecutionContext using credentials and create a new operation instance.
execution_context = ExecutionContext.create(credentials)

In [14]:
import logging

In [29]:

logging.basicConfig(level=os.environ.get("LOGLEVEL", "INFO"))

try:
  #get base path.
  #base_path = os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))
  extract_pdf_operation = ExtractPDFOperation.create_new()
  #Set operation input from a source file.
  source = FileRef.create_from_local_file(input_pdf)
  extract_pdf_operation.set_input(source)

  #Build ExtractPDF options and set them into the operation
  extract_pdf_options: ExtractPDFOptions = ExtractPDFOptions.builder() \
    .with_elements_to_extract([ExtractElementType.TEXT, ExtractElementType.TABLES]) \
    .with_element_to_extract_renditions(ExtractRenditionsElementType.TABLES) \
    .with_table_structure_format(TableStructureType.CSV) \
    .build()
  extract_pdf_operation.set_options(extract_pdf_options)

  #Execute the operation.
  result: FileRef = extract_pdf_operation.execute(execution_context)

  #Save the result to the specified location.
  result.save_as("ExtractTextTableWithTableStructure.zip")
except (ServiceApiException, ServiceUsageException, SdkException):
  logging.exception("Exception encountered while executing operation")


In [30]:
## Review the contents

In [59]:
import zipfile

def list_zip_files(zip_file_path):
    """
    Recursively lists all filenames within a given zip file.

    Args:
        zip_file_path (str): The path to the .zip file.
    """

    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        for info in zip_ref.infolist():
            if not info.is_dir():  # Check if it's a file (not a directory)
                print(info.filename)

# Example usage:
zip_file_path = "ExtractTextTableWithTableStructure.zip"  # Replace with your zip file path
list_zip_files(zip_file_path)

######
# Unzip the file

import zipfile

def extract_zip(zip_file_path, extract_to="."):
    """
    Extracts the contents of a zip file to a specified directory.

    Args:
        zip_file_path (str): The path to the zip file.
        extract_to (str, optional): The directory to extract the files into.
                                    Defaults to the current working directory (".")
    """
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(path=extract_to)

# Example usage:
zip_file_path = "ExtractTextTableWithTableStructure.zip"  # Replace with your zip file path
extract_to = "ExtractTextTableWithTableStructure"  # Optional: Specify an extraction directory

extract_zip(zip_file_path, extract_to)


#####


import json



def print_json_structure(data, level=0):
    """
    Prints the structure of the first 2 elements of a JSON object.

    Args:
        data: The JSON data.
        level: Indentation level for pretty-printing.
    """

    if level >= 2:  # Stop after printing 2 elements
        return

    if isinstance(data, dict):
        print("  " * level + str(list(data.keys())[0]) + ":")  # Print first key
        print_json_structure(data[list(data.keys())[0]], level + 1)  # Explore first value
        level -= 1  # Decrement level for next element

        if level >= 5:  # Stop after printing 2 elements
            return

        print("  " * level + str(list(data.keys())[1]) + ":")  # Print second key
        print_json_structure(data[list(data.keys())[1]], level + 1)  # Explore second value

    elif isinstance(data, list):
        print("  " * level + "-")  # Print list item indicator
        print_json_structure(data[0], level + 1)  # Explore first element

    else:
        print("  " * level + str(data))  # Print simple data type



# Example usage:
with open("ExtractTextTableWithTableStructure/structuredData.json", 'r') as f:  # Replace with your JSON file path
    json_data = json.load(f)

print_json_structure(json_data)




import os

def list_csv_files_os(directory):
    """Lists CSV files in a directory using the 'os' module."""
    csv_files = []
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            csv_files.append(filename)
    return csv_files

# Example usage:
directory = "ExtractTextTableWithTableStructure/tables"  # Replace with your folder path
csv_list = list_csv_files_os(directory)
print(csv_list)






structuredData.json
tables/fileoutpart0.csv
tables/fileoutpart29.png
tables/fileoutpart2.csv
tables/fileoutpart11.png
tables/fileoutpart18.csv
tables/fileoutpart21.png
tables/fileoutpart22.csv
tables/fileoutpart13.png
tables/fileoutpart10.csv
tables/fileoutpart8.csv
tables/fileoutpart1.png
tables/fileoutpart7.png
tables/fileoutpart15.png
tables/fileoutpart12.csv
tables/fileoutpart23.png
tables/fileoutpart19.png
tables/fileoutpart3.png
tables/fileoutpart28.csv
tables/fileoutpart27.png
tables/fileoutpart6.csv
tables/fileoutpart26.csv
tables/fileoutpart31.png
tables/fileoutpart17.png
tables/fileoutpart25.png
tables/fileoutpart14.csv
tables/fileoutpart20.csv
tables/fileoutpart5.png
tables/fileoutpart9.png
tables/fileoutpart30.csv
tables/fileoutpart16.csv
tables/fileoutpart24.csv
tables/fileoutpart4.csv
version:
  json_export:
page_segmentation:
  5
extended_metadata:
ID_instance:
  33 C8 F7 CF F8 B8 B2 11 0A 00 67 45 8B 6B C6 23 
ID_permanent:
39 34 20 30 31 20 44 43 20 43 46 20 46 38 20 4

In [55]:

## Parse JSON File to extract only text from Text data
text = " "
for item in json_data['elements']:
  try:
    text = text + " " + item['Text']
  except:
    pass

with open("ExtractedText.txt", "w", encoding="utf-8") as file:
        file.write(text)

In [67]:
## Prepare for Chunking

# 1.0 - Chunk Text files


with open("ExtractedText.txt") as f:
    Extracted_text = f.read()

#!pip install langchain
import langchain
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=100,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)


texts = text_splitter.create_documents([Extracted_text])
print(texts[0])
print(texts[1])


from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.document_loaders.csv_loader import UnstructuredCSVLoader


for csvfile in csv_list:
  loader = CSVLoader(file_path="ExtractTextTableWithTableStructure/tables/"+csvfile)
  #loader = UnstructuredCSVLoader(file_path="ExtractTextTableWithTableStructure/tables/"+csvfile, mode="elements")
  try:
    data = loader.load()
  except:
    print  ("Csv file not parsed : ",csvfile)
  #texts.append(data)
  #print (csvfile, data)

page_content='2023-12-06  Gemini: A Family of Highly Capable Multimodal Models  Gemini Team, Google 1  This'
page_content='Google 1  This report introduces a new family of multimodal models, Gemini, that exhibit remarkable'
Csv file not parsed :  fileoutpart4.csv


In [64]:
! pip install unstructured==0.5.6


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.4 MB/s eta 0:00:00
  Created wheel for unstructured: filename=unstructured-0.5.6-py3-none-any.whl size=1315700 sha256=b56bbb5ca4c8e29465fac0cc9bb1fbc84cdee56fd78da1079a7015d3984e29dc
  Stored in directory: /root/.cache/pip/wheels/8d/ba/38/39adebfeae4a6d48b6851964610929b716c66c861f7f6404f4
Successfully built unstructured
